# Data Handles (aka persistent identifiers)
- Each netcdf file should correspond to a UNIQUE handle, which is called the tracking_id in CMIP6-world.
- We can get information about these tracking_ids from the German Climate Computing Centre at http://handle-esgf.dkrz.de

### For example,
- for tracking_id = 'hdl:21.14100/2f67746f-580b-4750-a25f-ce2559d1c57b'

- We can cut and paste the tracking_id into [http://hdl.handle.net](http://hdl.handle.net) 
  this actually re-directs to [https://handle-esgf.dkrz.de/lp/21.14100/2f67746f-580b-4750-a25f-ce2559d1c57b](https://handle-esgf.dkrz.de/lp/21.14100/2f67746f-580b-4750-a25f-ce2559d1c57b) (try it out)
  
- This gives us the following information (also provides checksum and source_url):

        netcdf_file    = 'expc_Oyr_CESM2_historical_r11i1p1f1_gr_1850-1899.nc'
        file_size      = '440729165'  
        dataset_id     = 'hdl:21.14100/f03d7841-e21a-31fd-88c6-9632c0e2c5b1'  
        dataset_name   = 'CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.Oyr.expc.gr'
        version        = '20190514'

- If you now follow the link to the dataset_id, you see the following list of netcdf files and their corresponding tracking_ids, including the netcdf file with our given tracking_id (expc_Oyr_CESM2_historical_r11i1p1f1_gr_1850-1899.nc):

       expc_Oyr_CESM2_historical_r11i1p1f1_gr_1950-1999.nc	hdl:21.14100/9c92b764-4f3e-4aa2-9f4b-5a7612cf660f  
       expc_Oyr_CESM2_historical_r11i1p1f1_gr_2000-2014.nc	hdl:21.14100/405705f5-6566-4cad-83c1-07ce60e82fcb  
       expc_Oyr_CESM2_historical_r11i1p1f1_gr_1900-1949.nc	hdl:21.14100/eab727d0-c996-419f-8f38-1bd79641aca4  
       expc_Oyr_CESM2_historical_r11i1p1f1_gr_1850-1899.nc	hdl:21.14100/2f67746f-580b-4750-a25f-ce2559d1c57b  


- Note that this is a list of all netcdf files which were concatenated in time to make the complete dataset: 'CMIP6.CMIP.NCAR.CESM2.historical.r11i1p1f1.Oyr.expc.gr'

# Zarr store persistent identifier
- Each zarr store in our Cloud collection also has a tracking_id. This tracking id is the concatenation of tracking_ids for all netcdf file in the dataset.

### For example,
- For the zarr store = 'gs://cmip6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Oyr/expc/gr':

        ds = xr.open_zarr(fsspec.get_mapper('gs://cmip6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp370/r1i1p1f1/Amon/prc/gn'),consolidated=True)
        ds.attrs['tracking_id']) 
        ---
        'hdl:21.14100/2f67746f-580b-4750-a25f-ce2559d1c57b\nhdl:21.14100/eab727d0-c996-419f-8f38-1bd79641aca4\nhdl:21.14100/9c92b764-4f3e-4aa2-9f4b-5a7612cf660f\nhdl:21.14100/405705f5-6566-4cad-83c1-07ce60e82fcb'


# GOAL: Find version_id for each zarr store
Steps:
1. Get tracking_id from zarr store
2. Get version_id from tracking_id

Once we know the version_id for each zarr store, we can iterate through the catalog and replace old versions with new versions when they become available. 

- Unfortunately, the version_id is NOT contained in the netcdf file metadata, nor as part of the netcdf file's name.  Since we download the netcdf files (using GLOBUS or wget), the directory structure of the source machine is lost and therefore the version_id is lost.  (N.B., this is the ONLY DRS element which is not stored in the file's metadata) 
- Fortunately, using the Handle REST API, we can find the version_id corresponding to the constituent tracking_ids of each zarr store.
- Ideally, each netcdf file would have a UNIQUE tracking_id.  This is not always true (e.g., CESM assigns same tracking_id to multiple netcdf files).
- Ideally, each dataset consists of netcdf files with known tracking_ids.  This is not always true (e.g., the netcdf file was not properly registered)
- Ideally, each dataset corresponds to a single version_id.  This is not always true, either.
- Ideally, each zarr store has tracking_ids corresponding to the same dataset. This happens, for example, if the netcdf files from the ESGF search API were not all from the same version. (For example, v20191201 and v20191202). 
- If all of the above are true, than each zarr store would correspond to a unique version_id.  

### Step 1 - Get tracking_id from zarr store

In [ ]:
import zarr
import fsspec
import warnings

In [ ]:
zstore = 'gs://cmip6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Oyr/expc/gr'
mapper = fsspec.get_mapper(zstore)

group = zarr.open_consolidated(mapper)

zarr_tracking_id = group.attrs['tracking_id']

zarr_tracking_id

### Step 2 - Get version from tracking_id

In [ ]:
import requests

In [ ]:
client = requests.session()
baseurl =  'http://hdl.handle.net/api/handles/'
query1 = '?type=IS_PART_OF'
query2 = '?type=VERSION_NUMBER'

In [ ]:
# given a zarr store tracking_id, find the dataset tracking_id and then the version of the dataset

datasets = []
versions = []
names = []
for file_tracking_id in zarr_tracking_id.split('\n'):
    url1 = baseurl + file_tracking_id[4:]+query1
    r = client.get(url1)
    r.raise_for_status()
    dataset_tracking_id = r.json()['values'][0]['data']['value']
    datasets += [dataset_tracking_id]
    url2 = baseurl + dataset_tracking_id[4:] + query2
    r = client.get(url2)
    r.raise_for_status()
    versions += [r.json()['values'][0]['data']['value']]
    url = baseurl + dataset_tracking_id[4:]
    r = client.get(url)
    r.raise_for_status()
    names += [r.json()['values'][3]['data']['value']]

dataset_id = list(set(datasets)) 
version_id = list(set(versions))
dataset_name = list(set(names))
    
print('dataset_name = ',dataset_name,'\n')
print('version_id = ',version_id,'\n')
print('dataset_id = ', dataset_id,'\n')

In [ ]:
# Now, what if we try this for another? This example has two features. The tracking_ids of the netcdf files are not unique and this is a replacement version.
zstore = 'gs://cmip6/ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/ts/gn'
zarr_tracking_id = gsurl2tracks(zstore)
print(zarr_tracking_id)

In [ ]:
(ier, dataset_id,version_id,dataset_name) = tracks2version(zarr_tracking_id)
print('dataset_id = ',dataset_id)
print('version_id     = ',version_id)
print('dataset_name     = ',dataset_name)

ds_url = f'http://hdl.handle.net/{dataset_id}'
print('dataset_url    = ',ds_url)

In [ ]:
(ier, jdict_file, jdict_ds) = tracks2version(zarr_tracking_id, get_jdicts=True)
jdict_ds

In [ ]:
#In this case, there was an older version of this dataset:
old_tracking_id = 'hdl:21.14100/948e81df-eb28-4b31-92ec-828a4e2883cf' 
(ier, jdict_file, jdict_ds) = tracks2version(old_tracking_id,get_jdicts=True)
jdict_ds

In [ ]:
zstore = 'gs://cmip6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Oyr/expc/gr'
#zstore = 'gs://cmip6/ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/ts/gn'
zarr_tracking_id = gsurl2tracks(zstore)

(ier, surls) = tracks2source(zarr_tracking_id)
surls